In [2]:
import numpy as np 
import pandas as pd 
import daytime
import re 
from scipy.stats.stats import pearsonr
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import timeit
import pickle 
import scipy 
from cvxopt import matrix, solvers 
from keras.models import Sequential
from keras.layers import Dense
from keras import losses
from keras.optimizers import Adam
import tensorflow as tf

In [14]:
####################################################################################################################################
############# CLEAN THE DATASET TO EXTRACT TRAJECTORIES ############################################################################
####################################################################################################################################

def data_processing():
    print("Loading the raw data file... ")
    # IMPORTING THE RAW DATA OF SUBOPTIMAL POLICY
    df = pd.read_csv('raw_data.txt', sep='\t', engine='python', nrows=1000000, encoding= 'utf-16')
    print("/*************/")
    print("File loaded... ")

    # FUNCTIIONS USED TO CLEAN THE ABOVE RAW DATA 
    def read_data(file, enc='utf16', delim='\t', decim='.'):
        with open(file, newline='', encoding=enc) as datafile:
            return pd.read_csv(datafile, skiprows=0, header=0, delimiter=delim, decimal=decim, low_memory=False)


    def load_pkl(file):
        with open(file, 'rb') as inp:
            return pickle.load(inp)


    def save_pkl(var, file):
        with open(file, 'wb') as output:
            pickle.dump(var, output, pickle.HIGHEST_PROTOCOL)


    def running_mean(x, n):
        cumsum = np.cumsum(np.insert(x, 0, 0))
        return (cumsum[n:] - cumsum[:-n]) / float(n)

    def running_min(x, n):
        min_list = pd.DataFrame(x).rolling(n).min()
        return min_list.loc[(n-1):]

    def running_max(x, n):
        max_list = pd.DataFrame(x).rolling(n).max()
        return max_list.loc[(n-1):]

    def moving_avg_no_pad(pointList, winWidth):
        cumsum, moving_aves = [0], []
        #cumsum = [0] + list(accumulate(pointList))
        cumsum = np.cumsum(np.insert(pointList, 0, 0))
        for i, x in list(enumerate(pointList, 1)):
            if winWidth % 2 == 0:
                if i <= winWidth // 2:
                    moving_ave = (cumsum[i + winWidth // 2 - 1] - cumsum[0]) / (i + winWidth // 2 - 1)
                    moving_aves.append(moving_ave)
                elif i > len(pointList) - winWidth // 2 + 1:
                    moving_ave = (cumsum[len(pointList)] - cumsum[i - winWidth // 2 - 1]) / (len(pointList) - i + winWidth // 2 + 1)
                    moving_aves.append(moving_ave)
                else:
                    moving_ave = (cumsum[i + winWidth // 2 - 1] - cumsum[i - winWidth // 2 - 1]) / winWidth
                    moving_aves.append(moving_ave)
            else:
                if i <= winWidth // 2:
                    moving_ave = (cumsum[i + winWidth // 2] - cumsum[0]) / (i + winWidth // 2)
                    moving_aves.append(moving_ave)
                elif i > len(pointList) - winWidth // 2:
                    moving_ave = (cumsum[len(pointList)] - cumsum[i - winWidth // 2 - 1]) / (len(pointList) - i + winWidth // 2 + 1)
                    moving_aves.append(moving_ave)
                else:
                    moving_ave = (cumsum[i + winWidth // 2] - cumsum[i - winWidth // 2 - 1]) / winWidth
                    moving_aves.append(moving_ave)
        return moving_aves

    def range_sign(x, pos_thres=0):
        if x > pos_thres:
            return 1
        elif x < -pos_thres:
            return -1
        else:
            return 0


    def data_statistics_preprocess(df, target_column='Inc Electrical Antenna Tilt (deg)',drop_columns=[], pick_columns=[], split=0.7, reward_limit=20):
        df = df.drop(columns=['CGI'])
        # df = df.dropna(how='any')
        # Theoretical range of angle is considered as (0-25)
        df['Electrical Antenna Tilt (deg)'] = df['Electrical Antenna Tilt (deg)'].apply(
        lambda x: float(x) / 25)
        # Theoretical range of throughput is considered as (4,300,000 Kbps)
        df['Average UE PDCP DL Throughput (Kbps)'] = df['Average UE PDCP DL Throughput (Kbps)'].apply(
        lambda x: float(x) / 4300000)
        # Theoretical range of number of calls is considered as (3,200,000)
        df['Num Calls'] = df['Num Calls'].apply(
        lambda x: float(x) / 3200000)
        # Theoretical range of number of calls is considered as (-1 to 499)
        df['Time Advance Overshooting Factor'] = df['Time Advance Overshooting Factor'].apply(
        lambda x: float(x) / 500)
        # Theoretical range of number of calls is considered as (100)
        df['Inter Site Distance (Km)'] = df['Inter Site Distance (Km)'].apply(
        lambda x: float(x) / 100)

        df['Var Electrical Antenna Tilt (deg)'] = df['Var Electrical Antenna Tilt (deg)']

        # Theoretical range of number of calls is considered as (100)
        df['Inc Electrical Antenna Tilt (deg)'] = df ['Inc Electrical Antenna Tilt (deg)'].apply(
        lambda x: np.sign(x))
        # Percentages normalised by 100
        df['Low RSRP Samples Rate Edge (%)'] = df['Low RSRP Samples Rate Edge (%)'].apply(
        lambda x: float(x) / 100)
        df['Number of Times Interf (%)'] = df['Number of Times Interf (%)'].apply(
        lambda x: float(x) / 100)
        df['Number of Cells High Overlap High Rsrp Src Agg (%)'] = df['Number of Cells High Overlap High Rsrp Src Agg (%)']\
        .apply(lambda x: float(x) / 100)
        df['Number of Cells High Overlap High Rsrp Tgt Agg (%)'] = df['Number of Cells High Overlap High Rsrp Tgt Agg (%)']\
        .apply(lambda x: float(x) / 100)
        df['PDCCH CCE High Load (%)'] = df['PDCCH CCE High Load (%)'].apply(lambda x: float(x) / 100)

        df.reset_index(drop=True, inplace=True)
        return df


    def split_df(df, inputs=None, outputs=None):
        if outputs is None:
            raise Exception('Output cannot be None when splitting'.format())
        if inputs is None:
            return df.drop(outputs), df[outputs]
        return df[inputs], df[outputs]


    # OBTAIN THE CLEAN VERSION OF THE RAW DATA 
    clean_data = data_statistics_preprocess(df)


    print("columns:", clean_data.columns)


    cols = ["Electrical Antenna Tilt (deg)",
    "Average UE PDCP DL Throughput (Kbps)",
    "Num Calls",
    "Low RSRP Samples Rate Edge (%)",
    "FUZZY_LOW_RSRP_SAMPLES_EDGE_HIGH",
    "Number of Times Interf (%)",
    "FUZZY_NUM_TIMES_INTERF_HIGH",
    "Time Advance Overshooting Factor",
    "FUZZY_OSF_HIGH",
    "Number of Cells High Overlap High Rsrp Src Agg (%)",
    "FUZZY_NUM_CELLS_HIGH_OVERLAP_SRC_HIGH",
    "Number of Cells High Overlap High Rsrp Tgt Agg (%)",
    "FUZZY_NUM_TIMES_HIGH_OVERLAP_TGT_HIGH",
    "Inter Site Distance (Km)",
    "PDCCH CCE High Load (%)",
    "Inc Electrical Antenna Tilt (deg)",
    "Var Electrical Antenna Tilt (deg)",
    "Reward_Weighted_Delta_Driver_HighLevel_KPIs"]


    clean_data = clean_data[cols]
    clean_data = clean_data.dropna(how='any') 

    print("Shape of clean data:", clean_data.shape)


    # Create state space 
    def create_state(df):
        row_list = []
        row_list2 = []
        for index, rows in df.iterrows():
            my_list =[rows.iloc[1],
               rows.iloc[2],
               rows.iloc[3],
               rows.iloc[4],
               rows.iloc[5],
               rows.iloc[6],
               rows.iloc[7],
               rows.iloc[8],
               rows.iloc[9],
               rows.iloc[10],
               rows.iloc[11],
               rows.iloc[12],
               rows.iloc[13],
               rows.iloc[14]]
            row_list.append(my_list) 
            row_list2.append(rows.iloc[17])
        df['state'] = row_list
        df['action'] = row_list2
        rl_df = df[["state", "action", "Reward_Weighted_Delta_Driver_HighLevel_KPIs"]]
        rl_df = rl_df.rename(columns={ "Reward_Weighted_Delta_Driver_HighLevel_KPIs":"reward"})
        return rl_df


    newdf = create_state(clean_data)
    newdf = newdf.reset_index()
    newdf = newdf.drop(columns=['index'])
    print("Shape of new df:", newdf.shape)

    return newdf

In [28]:
####################################################################################################################################
#################### WRITE A DUMMY SIMULATOR #######################################################################################
####################################################################################################################################

NUM_OF_EPOCHS = 50

def simulator(newdf):
    # This simulator basically takes input as a state s and action a and spits out the next state s' i.e the new state is 'a' is taken in state 's'. 
    print("Creating the (s, a, r, s') pairs...")
    ls = []
    l = len(newdf)
    for index, row in newdf.iterrows():
        if index!= l-1:
            ls.append(newdf.iloc[index+1, 0])
        else:
            break

    newdf.drop(newdf.tail(1).index,inplace=True)
    newdf['next_state'] = ls

    print("Printing the new dataframe with (s, a, r, s') trajectories...")
    print("----------")
    print(newdf.columns)

    # Creating the training data 
    X = []
    for index, rows in newdf.iterrows():
        r = [rows[0], [rows[1]]]
        r = sum(r, [])
        X.append(r)	
    y = newdf.iloc[:,3]

    X_stack, y_stack = np.stack(X, axis=0), np.stack(y, axis=0)

    print("Creating the model for simulator...")
    model = Sequential()
    model.add(Dense(10, input_dim=len(newdf.iloc[0,0]) + 1, activation= 'relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(len(newdf.iloc[0,0])))
    model.compile(loss='logcosh', optimizer= Adam(lr= 0.0001))

    print("Started training...")
    model.fit(X_stack, y_stack, epochs=NUM_OF_EPOCHS, verbose=1)
    print("Simulator trained to predict s' from (s,a)...")

    model.save("transition_model_test.h5")

    return model


In [16]:
newdf = data_processing()

Loading the raw data file... 
/*************/
File loaded... 
columns: Index(['Date', 'ECell UserLabel', 'ENODEB Name', 'TAC', 'Colocated ECell',
       'Template', 'Rule', 'Azimuth', 'Latitude', 'Longitude',
       ...
       'RELAT_new_DELTA_PDCCH_HIGH_LOAD', 'RELAT_new_DELTA_RETAINABILITY_DCR',
       'REL_FUZZY_LOW_RSRP_EDGE', 'REL_FUZZY_OVERLAP_SRC',
       'REL_FUZZY_OVERLAP_TGT', 'REL_FUZZY_INTERF', 'REL_FUZZY_OSF',
       'RELAT_new_NUM_CALLS', 'Reward_Weighted_Delta_Driver_KPIs',
       'Reward_Weighted_Delta_Driver_HighLevel_KPIs'],
      dtype='object', length=356)
Shape of clean data: (239319, 18)
Shape of new df: (239319, 3)


In [17]:
def create_data(newdf):
	# This simulator basically takes input as a state s and action a and spits out the next state s' i.e the new state is 'a' is taken in state 's'. 
	print("Creating the (s, a, r, s') pairs...")
	ls = []
	l = len(newdf)
	for index, row in newdf.iterrows():
	    if index!= l-1:
	        ls.append(newdf.iloc[index+1, 0])
	    else:
	        break
	newdf.drop(newdf.tail(1).index,inplace=True)
	newdf['next_state'] = ls
	print("Printing the new dataframe with (s, a, r, s') trajectories...")
	print("----------")
	print(newdf.columns)
	return newdf

newdf = create_data(newdf)

Creating the (s, a, r, s') pairs...
Printing the new dataframe with (s, a, r, s') trajectories...
----------
Index(['state', 'action', 'reward', 'next_state'], dtype='object')


In [20]:
len(newdf.state[0])

14

In [29]:
transition_model = simulator(newdf)

Creating the (s, a, r, s') pairs...
Printing the new dataframe with (s, a, r, s') trajectories...
----------
Index(['state', 'action', 'reward', 'next_state'], dtype='object')
Creating the model for simulator...
Started training...
Epoch 1/50
239317/239317 [==============================] - 6s 26us/step - loss: 0.0134
Epoch 2/50
239317/239317 [==============================] - 6s 24us/step - loss: 0.0113
Epoch 3/50
239317/239317 [==============================] - 6s 24us/step - loss: 0.0112
Epoch 4/50
239317/239317 [==============================] - 6s 24us/step - loss: 0.0111
Epoch 5/50
239317/239317 [==============================] - 6s 24us/step - loss: 0.0111
Epoch 6/50
239317/239317 [==============================] - 6s 24us/step - loss: 0.0110
Epoch 7/50
239317/239317 [==============================] - 6s 24us/step - loss: 0.0110
Epoch 8/50
239317/239317 [==============================] - 6s 24us/step - loss: 0.0110
Epoch 9/50
239317/239317 [==============================] - 6s 2

In [62]:
stack_ = [newdf.state[0], [newdf.action[0]]]
stack_ = reduce(operator.concat, stack_)
print(stack_)
stack_ = np.asarray(stack_)
a = [stack_]
a = np.asarray(a)
res = transition_model.predict(a)

[0.0042143069767441865, 0.0431975, 0.21715, 0.0, 0.022640000000000004, 0.0, 0.00051, 0.0, 0.0034000000000000002, 0.0, 0.00618, 0.0, 0.026269999999999998, 0.0, 0.0203775593]


In [60]:
import os

if os.path.isfile("transition_model.h5"):
    print("hi")
else:
    print("ho")

hi


In [52]:
res

array([[3.3621565e-03, 5.6585886e-02, 5.4282612e-01, 4.7278780e-01,
        1.5290713e-02, 7.3866114e-02, 4.2882180e-03, 2.4689813e-01,
        3.3692457e-03, 1.6849816e-02, 3.2768613e-03, 1.2006344e-02,
        2.7062554e-02, 3.4150388e-04]], dtype=float32)

In [59]:
act = newdf.next_state[0]